# <font color=red><center>**Store recommendations in Azure Cosmos DB**</center></font>

**In ths notebook, we train our best model, run the predictions, and store the results in Azure Cosmos DB.**

**The recommendations are queried by the Azure Function consumed by Streamlit App.**

## **<font color=green>The project</font>**
"My Content" is a start-up who's goal is to encourage people to read by recommending relevant content to users.

In this project, we want to create a first MVP that will take the form of an application that will be recommend relevant articles to users based on their implicit preferences, their profiles and the articles content. This is known as a Recommender System. Thus, we will complete the following tasks: 

* We will compare different models of recommender system on the Globo.com dataset. 

* We will use Azure Functions to store the recommendations in Azure CosmosDB and to make the recommendations available to the users.

* We will integrate that Azure Functions into a Streamlit App that will be able to recommend relevant articles to users. 

## **<font color=green>Dataset</font>**
We will work with the Globo.com 

The dataset is composed of the following files :

- clicks/clicks/ : contains 385 CSV files
- clicks_hour_*.csv : contains one hour of clicks on the website
- articles_embeddings.pickle : pickle file containing the embeddings of the articles
- articles_metadata.csv : CSV file containing the metadata of the articles

# <font color=salmon>PART 1 - LIBRARIES, PARAMETERS</font>

## **<font color=green>Loading libraries</font>**

In [1]:
import os
import sys

# Add source directory to python path
sys.path.append(os.path.abspath("../"))

import azure.cosmos
from datetime import datetime
from pathlib import Path
from azure.cosmos import  CosmosClient, PartitionKey, exceptions

import pandas as pd
from implicit.als import AlternatingLeastSquares
from scipy import sparse
from tqdm import tqdm
from dotenv import load_dotenv
import pickle

# load_dotenv()
AZURE_COSMOSDB_URI = os.environ["AZURE_COSMOSDB_URI"]
AZURE_COSMOSDB_KEY = os.environ["AZURE_COSMOSDB_KEY"]

# Plotly as Pandas plotting backend
pd.options.plotting.backend = "plotly"


RAW_DATA_PATH = "../data"

CACHE = dict()

In [2]:
clicks = pd.concat(
    [
        pd.read_csv(
            click_file_path,
            parse_dates=["session_start", "click_timestamp"],
            date_parser=lambda x: datetime.fromtimestamp(int(int(x) / 1000)),
            dtype={
                "user_id": "category",
                "session_id": "category",
                "session_size": "int",
                "click_article_id": "category",
                "click_environment": "category",
                "click_deviceGroup": "category",
                "click_os": "category",
                "click_country": "category",
                "click_region": "category",
                "click_referrer_type": "category",
            },
        )
        for click_file_path in tqdm(
            sorted(Path(RAW_DATA_PATH, "clicks/clicks").glob("clicks_hour_*.csv"))
        )
    ],
    sort=False,
    ignore_index=True,
    verify_integrity=True,
)

clicks = clicks.astype(
    {"session_start": "datetime64[ns]", "click_timestamp": "datetime64[ns]"}
)

clicks.describe(include="all", datetime_is_numeric=True)

100%|████████████████████████████████████████████████████████████████████████████████| 385/385 [00:32<00:00, 11.94it/s]


,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
count,2988181,2988181,2988181,2.988181e+06,2988181,2988181,2988181,2988181,2988181,2988181,2988181,2988181
unique,322897,1048594,NaN,NaN,46033,NaN,3,5,8,11,28,7
top,5890,1507563657895091,NaN,NaN,160974,NaN,4,1,17,1,25,2
freq,1232,124,NaN,NaN,37213,NaN,2904478,1823162,1738138,2852406,804985,1602601
mean,NaN,NaN,2017-10-08 16:17:08.013155328,3.901885e+00,NaN,2017-10-08 16:51:05.070374400,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,2017-10-01 04:37:03,2.000000e+00,NaN,2017-10-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2017-10-04 15:35:52,2.000000e+00,NaN,2017-10-04 16:20:52,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2017-10-08 22:09:00,3.000000e+00,NaN,2017-10-08 22:35:30,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2017-10-11 21:16:54,4.000000e+00,NaN,2017-10-11 21:43:24,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,2017-10-17 05:36:19,1.240000e+02,NaN,2017-11-13 21:04:14,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
## Create implicit ratings

ratings = (
    clicks.reset_index()
    .groupby(["user_id", "click_article_id"])
    .agg(
        rating=("index", "count"),
    )
    .reset_index()
)
ratings

,user_id,click_article_id,rating
0,0,157541,1
1,0,160158,1
2,0,233470,1
3,0,313996,1
4,0,68866,1
...,...,...,...
2950705,99998,64329,1
2950706,99999,168784,1
2950707,99999,225055,1
2950708,99999,272143,1


In [5]:
## Create implicit ratings sparse matrix

ratings_sparse = sparse.csr_matrix(
    (
        ratings["rating"],
        (ratings["user_id"].astype("int"), ratings["click_article_id"].astype("int")),
    ),
)

ratings_sparse

<322897x364047 sparse matrix of type '<class 'numpy.int64'>'
	with 2950710 stored elements in Compressed Sparse Row format>

In [6]:
## Train ALS model

model = AlternatingLeastSquares()
model.fit(ratings_sparse)

  0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
## Test ALS model

model.recommend(11183, ratings_sparse[11183], N=5)

(array([160474, 119592,  96663,  64409, 108854]),
 array([0.26486713, 0.18765613, 0.12838125, 0.05783352, 0.05732467],
       dtype=float32))

In [8]:
# Save the model to disk
pickle.dump(model, open('model_implicit.pkl', 'wb'))

In [9]:
from azure.cosmos import  CosmosClient, PartitionKey #exceptions,

In [18]:
## Create CosmosDB client and database

client = CosmosClient(AZURE_COSMOSDB_URI, AZURE_COSMOSDB_KEY)
database = client.create_database_if_not_exists(id="Recommendation")
container = database.create_container_if_not_exists(
    id="UserArticles",
    partition_key=PartitionKey(path="/id"),
)

In [19]:
## Insert recommendations into CosmosDB

user_ids = ratings["user_id"].unique().astype("int")
recos, _ = model.recommend(user_ids, ratings_sparse[user_ids], N=5)

for i, user_id in tqdm(enumerate(user_ids)):
    container.upsert_item(
        body={
            "id": str(user_id),
            "articles": [str(article_id) for article_id in recos[i]],
        }
    )

322897it [3:13:50, 27.76it/s]


### Verification of the connexion

In [5]:
import logging
import azure.cosmos 
from azure.cosmos import CosmosClient
import azure.functions as func

# Check access to data from cosmos database.

database_name = "Recommendation",
articles_container_name = "UserArticles"

print('Connecting to Cosmos DB...')
client = CosmosClient(AZURE_COSMOSDB_URI, AZURE_COSMOSDB_KEY)
print('Connected.')

print('Connecting to database...')
database = client.get_database_client(database_name)
print('Connected.')

print('Connecting to articles container...')
articles_container = database.get_container_client(
                articles_container_name)
print('Connected.')

print('Ready!')

Connecting to Cosmos DB...
Connected.
Connecting to database...
Connected.
Connecting to articles container...
Connected.
Ready!
